In [ ]:
from jira import JIRA
import pandas as pd
import re
import sys
import os.path


def get_issues(server, project, df):
    options = {"server": server}
    jira = JIRA(options)
#     issue types are from link below 
#     https://www.tutorialspoint.com/jira/jira_issue_types.htm
#     https://confluence.atlassian.com/adminjiracloud/issue-types-844500742.html
#     jql='project='+project+" and issuetype in ('Bug','Improvement','New Feature','Sub-Task','Epic','Story','Task')"
    jql='project='+project+" and issuetype in ('Task', 'Sub-Task', 'Bug', 'Epic', 'Story', 'New Feature', 'Improvement')"
    
    block_size = 100
    block_num = 0
    while True:

        start_idx = block_num * block_size
        if block_num == 0:
            issues = jira.search_issues(jql, start_idx, block_size)
        else:
            more_issue = jira.search_issues(jql, start_idx, block_size)
            if len(more_issue)>0:
                for x in more_issue:
                    issues.append(x)
            else:
                break
        if len(issues) == 0:
            # Retrieve issues until there are no more to come
            break
        block_num += 1
        if len(issues)%1000 == 0:
            print(len(issues))
            
    print(f"--- there are {len(issues)} issues for {project}")
    
    for issue in issues:
        data = {}
        data['id'] = issue.raw['id']
        data['key'] = issue.raw['key']
        data['self'] = issue.raw['self']
        try:
            data['status_description'] = issue.raw['fields']['status']['description']
        except:
            data['status_description'] = ""
        try:
            data['status_name'] = issue.raw['fields']['status']['name']
        except:
            data['status_name'] = ""
        try:
            data['creator_name'] = issue.raw['fields']['creator']['name']
        except:
            data['creator_name'] = ""
        try:
            data['creator_displayName'] = issue.raw['fields']['creator']['displayName']
        except:
            data['creator_displayName'] = ""
        try:
            data['reporter_name'] = issue.raw['fields']['reporter']['name']
        except:
            data['reporter_name'] = ""
        try:
            data['reporter_displayName'] = issue.raw['feilds']['reporter']['displayName']
        except:
            data['reporter_displayName'] = ""
        try:
            data['issuetype_description'] = issue.raw['fields']['issuetype']['description']
        except:
            data['issuetype_description'] = ""
        try:
            data['issuetype_name'] = issue.raw['fields']['issuetype']['name']
        except:
            data['issuetype_name'] = ""
        try:
            data['created'] = issue.raw['fields']['created']
        except:
            data['created'] = ""
        try:
            data['updated'] = issue.raw['fields']['updated']
        except:
            data['updated'] = ""
        try:
            data['description'] = issue.raw['fields']['description']
        except:
            data['description'] = ""
        try:
            data['summary'] = issue.raw['fields']['summary']
        except:
            data['summary'] = ""
        try:
            data['assignee_name'] = issue.raw['feilds']['assignee']['name']
        except:
            data['assignee_name'] = ""
        try:
            data['duedate'] = issue.raw['fields']['duedate']
        except:
            data['duedate'] = ""
        try:
            versions_name = []
            for j in range(len(issue.raw['fields']['versions'])):
                versions_name.append(issue.raw['fields']['versions'][j]['name'])
            data['versions_name'] = versions_name
        except:
            data['versions_name'] = "" 
        try:
            versions_description = []
            for j in range(len(issue.raw['fields']['versions'])): 
                versions_description.append(issue.raw['fields']['versions'][j]['description'])
            data['versions_description'] = versions_description
        except:
            data['versions_description'] = ""
        try:
            versions_released = []
            for j in range(len(issue.raw['fields']['versions'])): 
                versions_released.append(issue.raw['fields']['versions'][j]['released'])
            data['versions_released'] = versions_released
        except:
            data['versions_released'] = ""
        try:
            versions_releasedDate = []
            for j in range(len(issue.raw['fields']['versions'])): 
                versions_releasedDate.append(issue.raw['fields']['versions'][j]['releasedDate'])
            data['versions_releasedDate'] = versions_releasedDate
        except:
            data['versions_releasedDate'] = ""
        try:
            fixVersions_description = []
            for j in range(len(issue.raw['fields']['fixVersions'])): 
                fixVersions_description.append(issue.raw['fields']['fixVersions'][j]['description'])
            data['fixVersions_description'] = fixVersions_description
        except:
            data['fixVersions_description'] = ""
        try:
            fixVersions_name = []
            for j in range(len(issue.raw['fields']['fixVersions'])): 
                fixVersions_name.append(issue.raw['fields']['fixVersions'][j]['name'])
            data['fixVersions_name'] = fixVersions_name
        except:
            data['fixVersions_name'] = ""  
        try:
            data['fixVersions_id'] = issue.raw['fields']['fixVersions']['id']
        except:
            data['fixVersions_id'] = ""
        try:
            data['fixVersions_archived'] = issue.raw['fields']['fixVersions']['archived']
        except:
            data['fixVersions_archived'] = ""
        try:
            data['fixVersions_released'] = issue.raw['fields']['fixVersions']['released']
        except:
            data['fixVersions_released'] = ""
        try:
            data['resolution_id'] = issue.raw['fields']['resolution']['id']
        except:
            data['resolution_id'] = ""
        try:
            data['resolution_description'] = issue.raw['fields']['resolution']['description']
        except:
            data['resolution_description'] = ""
        try:
            data['resolution_name'] = issue.raw['fields']['resolution']['name']
        except:
            data['resolution_name'] = ""
        try:
            data['priority_name'] = issue.raw['fields']['priority']['name']
        except:
            data['priority_name'] = ""
        try:
            data['priority_id'] = issue.raw['fields']['priority']['id']
        except:
            data['priority_id'] = ""
        try:
            data['labels'] = issue.raw['fields']['labels']['id']
        except:
            data['labels'] = ""
        try:
            data['assignee_displayName'] = issue.raw['feilds']['assignee']['displayName']
        except:
            data['assignee_displayName'] = ""
        try:
            data['assignee_active'] = issue.raw['feilds']['assignee']['active']
        except:
            data['assignee_active'] = ""  
        try:
            components_name = []
            for j in range(len(issue.raw['fields']['components'])): 
                components_name.append(issue.raw['fields']['components'][j]['name'])
            data['components_name'] = components_name
        except:
            data['components_name'] = ""
        try:
            components_description = []
            for j in range(len(issue.raw['fields']['components'])): 
                components_description.append(issue.raw['fields']['components'][j]['description'])
            data['components_description'] = components_description
        except:
            data['components_description'] = ""
        
        try:
            data['aggregateprogress_progress'] = issue.raw['feilds']['aggregateprogress']['progress']
        except:
            data['aggregateprogress_progress'] = "" 
        try:
            data['aggregateprogress_total'] = issue.raw['feilds']['progress']['total']
        except:
            data['aggregateprogress_total'] = "" 
        try:
            data['progress_progress'] = issue.raw['feilds']['progress']['total']
        except:
            data['progress_progress'] = "" 
        try:
            data['votes_votes'] = issue.raw['feilds']['votes']['votes']
        except:
            data['votes_votes'] = "" 
        try:
            data['votes_hasVoted'] = issue.raw['feilds']['votes']['hasVoted']
        except:
            data['votes_hasVoted'] = "" 
        try:
            data['issuetype_subtask'] = issue.raw['feilds']['issuetype']['subtask']
        except:
            data['issuetype_subtask'] = "" 
        try:
            data['project_key'] = issue.raw['feilds']['project']['key']
        except:
            data['project_key'] = "" 
        try:
            data['project_name'] = issue.raw['feilds']['project']['name']
        except:
            data['project_name'] = "" 
        try:
            data['project_projectTypeKey'] = issue.raw['feilds']['project']['projectTypeKey']
        except:
            data['project_projectTypeKey'] = ""
        try:
            data['project_projectCategory_description'] = issue.raw['feilds']['project']['projectCategory']['description']
        except:
            data['project_projectCategory_description'] = ""
        try:
            data['project_projectCategory_name'] = issue.raw['feilds']['project']['projectCategory']['name']
        except:
            data['project_projectCategory_name'] = ""
        try:
            data['resolutiondate'] = issue.raw['feilds']['resolutiondate']
        except:
            data['resolutiondate'] = ""
        try:
            data['workratio'] = issue.raw['feilds']['workratio']
        except:
            data['workratio'] = ""
        try:
            data['environment'] = issue.raw['feilds']['environment']
        except:
            data['environment'] = ""
            
        
        df = df.append(data, ignore_index=True)
    return df


sources = [("project server","project name")]

for source in sources:
    if not os.path.exists(source[1]+'_issues_data.csv'):
        print(f'getting issues for {source[1]} is started')
        
        cols = ['id', 'key', 'self', 'versions_name', 'status_description', 'status_name', 'creator_name', 'reporter_name', 
                'issuetype_description','issuetype_name', 'created', 'updated', 'description', 'summary', 'assignee_name', 'reporter_displayName',
                'duedate', 'fixVersions_description', 'fixVersions_name', 'fixVersions_id', 'fixVersions_archived', 'fixVersions_released', 'resolution_id',
                'resolution_description', 'resolution_name', 'priority_name', 'priority_id', 'labels', 'versions_description', 'versions_released',
                'versions_releasedDate', 'assignee_displayName', 'assignee_active', 'components_name', 'components_description', 'creator_displayName', 'aggregateprogress_progress', 
                'aggregateprogress_total', 'progress_progress', 'progress_total', 'votes_votes', 'votes_hasVoted', 'issuetype_subtask', 'project_key', 'project_name', 'project_projectTypeKey',
                'project_projectCategory_description', 'project_projectCategory_name', 'resolutiondate', 'workratio', 'environment']
        df = pd.DataFrame(columns=cols)
        
        df['versions_name'] = df['versions_name'].astype(object)
        df['versions_description'] = df['versions_description'].astype(object)
        df['versions_released'] = df['versions_released'].astype(object)
        df['versions_releasedDate'] = df['versions_releasedDate'].astype(object)
        df['fixVersions_description'] = df['fixVersions_description'].astype(object)
        df['components_name'] = df['components_name'].astype(object)
        df['components_description'] = df['components_description'].astype(object)
        df['fixVersions_name'] = df['fixVersions_name'].astype(object)
        
        repo_dataframe = get_issues(source[0], source[1], df)
        print(f'data frame shape is : {repo_dataframe.shape}')
        repo_dataframe.to_csv(source[1]+'_issues_data.csv', index=False)
        print(f"issues for {source[1]} saved succesfully!!! ;)")


